In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# knapsack problem
# items: water bottle, snack, laptop, camera, rain jacket
# weight is measured in pounds, volume is square inches, value is on a scale from 1 to 10 of importance for a given situation

class item:
    def __init__(self, name, weight, volume, value):
        self.name = name
        self.weight = weight
        self.volume = volume
        self.value = value
        
class phenotype:
    def __init__(self,items,weight,volume,value):
        self.items = items
        self.weight = weight
        self.volume = volume
        self.value = value

In [ ]:
water_bottle = item("water bottle",4,6,10)
snack = item("snack",3,5,8)
laptop = item("laptop",5,5,3)
camera = item("camera",2,2,8)
rain_jacket = item("rain jacket",1,1,5)
gun = item("pistol",6,4,8)
item_list = [water_bottle,snack,laptop,camera,rain_jacket,gun]

In [ ]:
# since there are 5 items, each individual's chromosome will be a binary of length 5

def random_chromosome(chrom_shape):
    chromosome = np.random.randint(0,2,chrom_shape)
    return chromosome

def chromosome_phenotype(chromosome,chrom_shape):
    chrom_items = []
    chrom_weight = []
    chrom_volume = []
    chrom_value = []
    for i in range(chrom_shape[0]):
        if chromosome[i] == 1:
            chrom_items.append(item_list[i].name)
            chrom_weight.append(item_list[i].weight)
            chrom_volume.append(item_list[i].volume)
            chrom_value.append(item_list[i].value)
    chrom_pheno = phenotype(chrom_items,np.sum(chrom_weight),
                           np.sum(chrom_volume),np.sum(chrom_value))
    return chrom_pheno

def valid_solution(phenotype, max_weight, max_volume):
    valid = True
    if phenotype.weight > max_weight:
        valid = False
    if phenotype.volume > max_volume:
        valid = False
    return valid

def select_chromosome(population,chrom_shape):
    chromosome = np.empty(chrom_shape)
    chromosome[:,0] = population[:,np.random.randint(0,population.shape[1])]
    return chromosome

def init_population(pop_size,chrom_shape,max_weight,max_volume):
    population = np.empty(pop_size)
    i = 0
    while i < pop_size[1]:
        chromosome = random_chromosome(chrom_shape)
        phenotype = chromosome_phenotype(chromosome,chrom_shape)
        valid = valid_solution(phenotype,max_weight,max_volume)
        if valid == True:
            population[:,i] = chromosome[:,0]
            i = i+1
    return population

def tournament_selection(population,chrom_shape):
    chrom_1 = select_chromosome(population,chrom_shape)
    phenotype_1 = chromosome_phenotype(chrom_1,chrom_shape)
    chrom_2 = select_chromosome(population,chrom_shape)
    phenotype_2 = chromosome_phenotype(chrom_2,chrom_shape)
    if phenotype_1.value > phenotype_2.value:
        winner = chrom_1
    if phenotype_1.value < phenotype_2.value:
        winner = chrom_2
    if phenotype_1.value == phenotype_2.value:
        rand_choice = np.random.randint(0,2)
        if rand_choice == 0:
            winner = chrom_1
        else:
            winner = chrom_2
    return winner 

def crossover(p1, p2):
    cross_point = np.random.randint(0,len(p1))
    split1 = p1[0:cross_point]
    split2 = p1[cross_point:len(p1)]
    child = np.concatenate((split1,split2),axis=0)
    return child
    
def mutation(chromosome):
    mut_index_1 = np.random.randint(0,len(chromosome))
    mut_index_2 = np.random.randint(0,len(chromosome))
    a = chromosome[mut_index_1,0]
    chromosome[mut_index_1,0] = chromosome[mut_index_2,0]
    chromosome[mut_index_2] = a
    return chromosome

def next_generation(population,mut_rate,pop_size,max_weight,max_volume,chrom_shape):
    
    new_gen = np.empty(pop_size)
    j = 0
    while j < pop_size[1]:
        parent_1 = tournament_selection(population,chrom_shape)
        parent_2 = tournament_selection(population,chrom_shape)
        child = crossover(parent_1,parent_2)
        child_phenotype = chromosome_phenotype(child,chrom_shape)
        if np.random.random() < mut_rate:
            child = mutation(child)
        
        if valid_solution(child_phenotype,max_weight,max_volume) == True:
            new_gen[:,j] = child[:,0]
            j = j +1
    
    return new_gen

def get_best_solution(population,pop_size,chrom_shape):
    best_solution = chromosome_phenotype(population[:,0],chrom_shape)
    for k in range(1,pop_size[1]):
        next_solution = chromosome_phenotype(population[:,k],chrom_shape)
        if next_solution.value > best_solution.value:
            best_solution = next_solution
    return best_solution
            
def genetic_algorithm(pop_size,chrom_shape,num_gens,mut_rate,max_weight,max_volume):
    all_solutions = []
    pop = init_population(pop_size,chrom_shape,max_weight,max_volume)
    best_solution = get_best_solution(pop,pop_size,chrom_shape)
    print("generation: 1")
    print("best solution: ",best_solution.items)
    print("value: ",best_solution.value)
    print("weigh: ",best_solution.weight)
    print("volume: ",best_solution.volume)
    print("-------------------------------")
    all_solutions.append(best_solution)
    for l in range(num_gens):
        current_pop = next_generation(pop,mut_rate,pop_size,max_weight,max_volume,chrom_shape)
        best_solution = get_best_solution(current_pop,pop_size,chrom_shape)
        all_solutions.append(best_solution)
        print("generation: ",l+2)
        print("best solution: ",best_solution.items)
        print("value: ",best_solution.value)
        print("weigh: ",best_solution.weight)
        print("volume: ",best_solution.volume)
        print("-------------------------------")
    return all_solutions
        
        

In [ ]:
max_weight = 16
max_volume = 16
chrom_shape = (5,1)
pop_size = (5,20)
mut_rate = 0.2
num_gens = 40

In [ ]:
all_solutions = genetic_algorithm(pop_size,chrom_shape,num_gens,mut_rate,max_weight,max_volume)

In [ ]:
best_value = 0
for solution in all_solutions:
    if solution.value > best_value:
        best_overall = solution
        best_items = solution.items
        best_weight = solution.weight
        best_volume = solution.volume
        best_value = solution.value

In [ ]:
print("best solution: ",best_items)
print("value: ",best_value)
print("weigh: ",best_weight)
print("volume: ",best_volume)